In [11]:
import sys
sys.path.append('../../UCI_experiments/')

import torch
from model.metrics import MAE, MSE, RMSE, get_coeff, NLL
import numpy as np

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

from uncertainty_toolbox.metrics_calibration import miscalibration_area

In [12]:
import torch

def NLL(input: torch.Tensor, target: torch.Tensor, var: torch.Tensor, full=True, eps=1e-06):
    return torch.nn.functional.gaussian_nll_loss(input.flatten(), target.flatten(), var.flatten(), full=full, eps=eps)

def RMSE(input, target):
    return torch.sqrt(torch.mean((input.flatten() - target.flatten())**2))

def MSE(input, target):
    return torch.mean((input.flatten() - target.flatten())**2)

def MAE(input, target):
    return torch.mean(torch.abs((input.flatten() - target.flatten())))

#use zero eps everywhere -> was not necessary during training, so no addition should be necessary either
def get_coeff(input: torch.Tensor, target: torch.Tensor, var: torch.Tensor) -> torch.tensor:
    """ Returns dimensionless NLL coefficient
    """
    
    mse = MSE(input,target)
    uncertainty_estimate = (input.flatten() - target.flatten())**2
    
    LL_best = torch.nn.functional.gaussian_nll_loss(input.flatten(), target.flatten(), uncertainty_estimate.flatten(), full=False, eps=0.)
    
    LL_worst_case_best_RMSE = torch.nn.functional.gaussian_nll_loss\
        (input.flatten(), target.flatten(), torch.ones_like(var.flatten())*mse, full=False, eps=0.)
    
    LL_actual = torch.nn.functional.gaussian_nll_loss(input.flatten(), target.flatten(), var.flatten(), full=False, eps=0.)
    
    coeff = 1/( LL_best - LL_worst_case_best_RMSE) * (LL_actual - LL_worst_case_best_RMSE) * 100

    return coeff

In [13]:
hartree_to_ev = 27.21138602


for identifier in ["H2O","LiPS","BaTiO3","QM9"]:
    #shallow_ens refers to NLL opt: H2O, LiPS and CRPS opt for BaTiO3 and QM9
    for architecture in ["shallow_ens","mse_ens"]:
        test_energy = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/test_energy.pt').detach().numpy().flatten()
        test_pred_energy = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/test_pred_energy.pt').detach().numpy().flatten()
        #test_pred_forces = torch.load(f'./{identifier}/{architecture}/test_pred_forces.pt').detach().numpy()
        test_pred_energy_var = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/test_pred_energy_var.pt').detach().numpy().flatten()


        # now for validation 
        val_energy = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/val_energy.pt').detach().numpy().flatten()
        val_pred_energy = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/val_pred_energy.pt').detach().numpy().flatten()
        #val_pred_forces = torch.load(f'./{identifier}/{architecture}/val_pred_forces.pt').detach()
        val_pred_energy_var = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/val_pred_energy_var.pt').detach().numpy().flatten()

        if identifier == "QM9":
            test_energy *= hartree_to_ev
            test_pred_energy *= hartree_to_ev
            test_pred_energy_var *= hartree_to_ev**2
            val_energy *= hartree_to_ev
            val_pred_energy *= hartree_to_ev
            val_pred_energy_var *= hartree_to_ev**2

        z_val_split = np.abs(val_energy-val_pred_energy)
        alpha = np.sqrt(np.mean(z_val_split**2/val_pred_energy_var, axis=0))

        z = np.abs(test_energy- test_pred_energy)

        mae = mean_absolute_error(test_energy, test_pred_energy)
        mse = mean_squared_error(test_energy, test_pred_energy)
        rmse = mean_squared_error(test_energy, test_pred_energy,squared=False)

        coef =  get_coeff(torch.tensor(test_energy), torch.tensor(test_pred_energy), torch.tensor(test_pred_energy_var))
        coef_cal = get_coeff(torch.tensor(test_energy), torch.tensor(test_pred_energy), torch.tensor(test_pred_energy_var)*alpha**2)

        ma = miscalibration_area(test_pred_energy.reshape(-1,1), np.sqrt(test_pred_energy_var).reshape(-1,1), test_energy.reshape(-1,1))
        ma_cal = miscalibration_area(test_pred_energy.reshape(-1,1), np.sqrt(test_pred_energy_var*alpha**2).reshape(-1,1), test_energy.reshape(-1,1))
        
        nll = NLL(torch.tensor(test_energy), torch.tensor(test_pred_energy), torch.tensor(test_pred_energy_var))
        nll_cal = NLL(torch.tensor(test_energy), torch.tensor(test_pred_energy), torch.tensor(test_pred_energy_var)*alpha**2)

        print(f'{identifier} {architecture} MAE: {mae:.4f} RMSE: {rmse:.4f} alpha: {alpha:.4f}, coeff: {coef:.4f}, coeff calibrated: {coef_cal:.4f}, NLL: {nll:.4f}, NLL calibrated: {nll_cal:.4f}')
        print(f'{identifier} {architecture} MA: {ma:.4f}, MA calibrated: {ma_cal:.4f}')


H2O shallow_ens MAE: 0.2181 RMSE: 0.4127 alpha: 1.1090, coeff: 59.7128, coeff calibrated: 58.9628, NLL: -0.4135, NLL calibrated: -0.4016
H2O shallow_ens MA: 0.0127, MA calibrated: 0.0418
H2O mse_ens MAE: 0.2305 RMSE: 0.3689 alpha: 13.8017, coeff: -4610.7609, coeff calibrated: 33.0798, NLL: 58.9034, NLL calibrated: 0.0021
H2O mse_ens MA: 0.4064, MA calibrated: 0.0897
LiPS shallow_ens MAE: 0.0863 RMSE: 0.2134 alpha: 1.3461, coeff: 55.2924, coeff calibrated: 63.6933, NLL: -1.1814, NLL calibrated: -1.3417
LiPS shallow_ens MA: 0.0771, MA calibrated: 0.0245
LiPS mse_ens MAE: 0.0920 RMSE: 0.1784 alpha: 4.1100, coeff: -409.8720, coeff calibrated: 58.0604, NLL: 5.4174, NLL calibrated: -1.1153
LiPS mse_ens MA: 0.3543, MA calibrated: 0.0126
BaTiO3 shallow_ens MAE: 0.0073 RMSE: 0.0122 alpha: 0.4928, coeff: 11.8515, coeff calibrated: 36.6664, NLL: -3.1250, NLL calibrated: -3.4076
BaTiO3 shallow_ens MA: 0.2099, MA calibrated: 0.0465
BaTiO3 mse_ens MAE: 0.0065 RMSE: 0.0096 alpha: 3.3024, coeff: -428.

In [14]:
hartree_to_ev = 27.21138602


for identifier in ["BaTiO3","QM9"]:
    for architecture in ["shallow_ens_nll"]:
        test_energy = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/test_energy.pt').detach().numpy().flatten()
        test_pred_energy = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/test_pred_energy.pt').detach().numpy().flatten()
        #test_pred_forces = torch.load(f'./{identifier}/{architecture}/test_pred_forces.pt').detach().numpy()
        test_pred_energy_var = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/test_pred_energy_var.pt').detach().numpy().flatten()


        # now for validation 
        val_energy = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/val_energy.pt').detach().numpy().flatten()
        val_pred_energy = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/val_pred_energy.pt').detach().numpy().flatten()
        #val_pred_forces = torch.load(f'./{identifier}/{architecture}/val_pred_forces.pt').detach()
        val_pred_energy_var = torch.load(f'../../Atomistic_experiments/materials_model_predictions/{identifier}/{architecture}/val_pred_energy_var.pt').detach().numpy().flatten()

        if identifier == "QM9":
            test_energy *= hartree_to_ev
            test_pred_energy *= hartree_to_ev
            test_pred_energy_var *= hartree_to_ev**2
            val_energy *= hartree_to_ev
            val_pred_energy *= hartree_to_ev
            val_pred_energy_var *= hartree_to_ev**2

        z_val_split = np.abs(val_energy-val_pred_energy)
        alpha = np.sqrt(np.mean(z_val_split**2/val_pred_energy_var, axis=0))

        z = np.abs(test_energy- test_pred_energy)

        mae = mean_absolute_error(test_energy, test_pred_energy)
        mse = mean_squared_error(test_energy, test_pred_energy)
        rmse = mean_squared_error(test_energy, test_pred_energy,squared=False)

        coef =  get_coeff(torch.tensor(test_energy), torch.tensor(test_pred_energy), torch.tensor(test_pred_energy_var))
        coef_cal = get_coeff(torch.tensor(test_energy), torch.tensor(test_pred_energy), torch.tensor(test_pred_energy_var)*alpha**2)

        ma = miscalibration_area(test_pred_energy.reshape(-1,1), np.sqrt(test_pred_energy_var).reshape(-1,1), test_energy.reshape(-1,1))
        ma_cal = miscalibration_area(test_pred_energy.reshape(-1,1), np.sqrt(test_pred_energy_var*alpha**2).reshape(-1,1), test_energy.reshape(-1,1))
        
        nll = NLL(torch.tensor(test_energy), torch.tensor(test_pred_energy), torch.tensor(test_pred_energy_var))
        nll_cal = NLL(torch.tensor(test_energy), torch.tensor(test_pred_energy), torch.tensor(test_pred_energy_var)*alpha**2)

        

        print(f'{identifier} {architecture} MAE: {mae:.4f} RMSE: {rmse:.4f} alpha: {alpha:.4f}, coeff: {coef:.4f}, coeff calibrated: {coef_cal:.4f}, NLL: {nll:.4f}, NLL calibrated: {nll_cal:.4f}')
        print(f'{identifier} {architecture} MA: {ma:.4f}, MA calibrated: {ma_cal:.4f}')


BaTiO3 shallow_ens_nll MAE: 0.0100 RMSE: 0.0209 alpha: 0.7773, coeff: -29.4067, coeff calibrated: -34.3042, NLL: -2.1349, NLL calibrated: -2.3409
BaTiO3 shallow_ens_nll MA: 0.3146, MA calibrated: 0.2881
QM9 shallow_ens_nll MAE: 0.0259 RMSE: 0.0529 alpha: 6.1363, coeff: -1089.9793, coeff calibrated: 1.9791, NLL: 12.1404, NLL calibrated: -1.5460
QM9 shallow_ens_nll MA: 0.2524, MA calibrated: 0.1836
